PROYECTO

Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [9]:
agri_df = pd.read_csv("/workspaces/proyectfinaltour/data/processed/unificado/agricola.csv", encoding='latin1')
weather_df = pd.read_csv("/workspaces/proyectfinaltour/data/processed/unificado/weather.csv")

In [10]:
weather_df.head()

,city,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,moonrise,...,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,Barcelona,2008-07-01,27,22,0.0,14.5,6,6,4,04:04 AM,...,25,19,8,63,0.0,1015,27,10,248,12
1,Barcelona,2008-07-02,25,19,0.0,14.5,6,6,0,05:07 AM,...,24,11,6,78,0.0,1012,25,10,218,9
2,Barcelona,2008-07-03,25,20,0.0,14.5,6,6,0,06:21 AM,...,23,14,22,73,0.0,1012,25,10,164,11
3,Barcelona,2008-07-04,25,19,0.0,14.5,6,6,3,07:41 AM,...,22,22,5,68,0.0,1017,25,10,186,16
4,Barcelona,2008-07-05,27,20,0.0,14.5,6,6,10,08:59 AM,...,24,18,5,74,0.0,1014,27,10,232,14


In [11]:
agri_df.head(8)

,Ciudad,Cultivo,year,Mes,Kilos,HectÃ¡reas
0,Bilbao,Olivo,2008,1,16795,475.850010
1,Bilbao,Olivo,2008,2,45732,302.456577
2,Bilbao,Olivo,2008,3,7265,86.437315
3,Bilbao,Olivo,2008,4,38194,235.031957
4,Bilbao,Olivo,2008,5,45131,80.004741
5,Bilbao,Olivo,2008,6,42090,20.086402
6,Bilbao,Olivo,2008,7,1769,363.779398
7,Bilbao,Olivo,2008,8,3433,99.094234


# Paso 2: Preparación de los Datos
Normalizar los nombres de las columnas
Para facilitar la manipulación de los datos, vamos a normalizar los nombres de las columnas.

In [12]:
# Normalizar los nombres de las columnas
agri_df.columns = agri_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

# Mostrar las primeras filas del dataframe para verificar
agri_df.head()


,ciudad,cultivo,year,mes,kilos,hectã¡reas
0,Bilbao,Olivo,2008,1,16795,475.850010
1,Bilbao,Olivo,2008,2,45732,302.456577
2,Bilbao,Olivo,2008,3,7265,86.437315
3,Bilbao,Olivo,2008,4,38194,235.031957
4,Bilbao,Olivo,2008,5,45131,80.004741


In [23]:
print(agri_df.columns)

Index(['Ciudad', 'Cultivo', 'year', 'Mes', 'Kilos', 'Hectáreas'], dtype='object')


2.1 - Integración de Datos
Ya hemos cargado los datos agrícolas y meteorológicos. Vamos a proceder con la integración de estos datos.

Ajuste y Unión de Datos
Crear Fechas de Índice en el Dataset Agrícola
Vamos a crear una columna de fechas en el dataset agrícola para alinearlo con el dataset meteorológico.

In [25]:
# Asegurarse de que las columnas 'year' y 'Mes' están en formato correcto
agri_df['year'] = agri_df['year'].astype(int)
agri_df['Mes'] = agri_df['Mes'].astype(int)

# Renombrar la columna 'Mes' a 'month' temporalmente
agri_df = agri_df.rename(columns={'Mes': 'month'})

# Crear la columna 'date' usando el día 15 del mes correspondiente
agri_df['date'] = pd.to_datetime(agri_df[['year', 'month']].assign(day=15))

# Restaurar el nombre original de la columna 'month' a 'Mes'
agri_df = agri_df.rename(columns={'month': 'Mes'})

# Mostrar las primeras filas para verificar
print(agri_df.head())


   Ciudad Cultivo  year  Mes  Kilos   Hectáreas       date
0  Bilbao   Olivo  2008    1  16795  475.850010 2008-01-15
1  Bilbao   Olivo  2008    2  45732  302.456577 2008-02-15
2  Bilbao   Olivo  2008    3   7265   86.437315 2008-03-15
3  Bilbao   Olivo  2008    4  38194  235.031957 2008-04-15
4  Bilbao   Olivo  2008    5  45131   80.004741 2008-05-15


In [26]:
output_path = '/workspaces/proyectfinaltour/data/processed/datasets/agricola_procesado.csv'

# Guardar el DataFrame en un archivo CSV en el directorio especificado
agri_df.to_csv(output_path, index=False)